In [1]:
# import package
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, ConcatDataset
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.utils as vutils
from torchvision.datasets import DatasetFolder 
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from RealESRGAN import RealESRGAN
import os
import torchattacks
from models_c import *
from tqdm import tqdm
import cv2

noise_color = 'w'

In [2]:
# Preprocessing

test_transforms = transforms.Compose([#transforms.RandomCrop((256, 256)),
                                       transforms.CenterCrop(224), 
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

toPillow = transforms.ToPILImage()
totensor = transforms.ToTensor()

In [3]:
model = models.densenet201(weights='DenseNet201_Weights.DEFAULT')
model = nn.Sequential(*list(model.children())[:-1],ClassifierH2())
model = model.to(device)

#checkpoint = torch.load(f'./pretrain/flower102_2023052039.pt')
checkpoint = torch.load(f'./output_model_{noise_color}/flower102_2023052048_{noise_color}.pt')
model.load_state_dict(checkpoint)
model.eval()  
print('load model : flower102')

load model : flower102


In [4]:
clean_set = DatasetFolder(f'flower102_test_set_{noise_color}', loader=lambda x: Image.open(x), extensions="jpg", transform = test_transforms)
adv_set = DatasetFolder(f'flower_test_adv_set_{noise_color}', loader=lambda x: Image.open(x), extensions="jpg", transform = test_transforms)
test_set = ConcatDataset([clean_set, adv_set])
#test_set = DatasetFolder(f'flower102/test', loader=lambda x: Image.open(x), extensions="jpg", transform = test_transforms)

test_loader = DataLoader(test_set, batch_size=128, shuffle=True, num_workers=0)

Debugger :  class -> idx 
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '80': 80, '81': 81, '82': 82, '83': 83, '84': 84, '85': 85, '86': 86, '87': 87, '88': 88, '89': 89, '90': 90, '91': 91, '92': 92, '93': 93, '94': 94, '95': 95, '96': 96, '97': 97, '98': 98, '99

In [5]:
total = 0
accuracy = 0
criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for batch in tqdm(test_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
            
        total += imgs.shape[0]
            
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        #print(preds)
        #print(labels)
        loss = criterion(outputs, labels)
        accuracy += torch.sum(preds == labels.data)
print(accuracy)
print(total)
print(f'Accuracy  = {accuracy / total}')

100%|██████████| 13/13 [00:02<00:00,  4.62it/s]


tensor(1202, device='cuda:0')
1638
Accuracy  = 0.7338217496871948
